In [20]:
import numpy as np
import pandas as pd
import json
import gzip
import ast

In [21]:
archivo = 'C:/Users/HP/Downloads/user_reviews.json.gz'

data = []


with gzip.open(archivo, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data= ast.literal_eval(line)
            data.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Crear un DataFrame a partir de los datos JSON
data = pd.DataFrame(data)

In [22]:
data.drop('user_url', axis=1, inplace=True)

In [23]:
data.head()

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."


In [24]:
data_reviews = data.explode('reviews').reset_index(drop=True)

In [25]:
data_reviews.head(20)

,user_id,reviews
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
5,js41637,"{'funny': '', 'posted': 'Posted November 29, 2..."
6,evcentric,"{'funny': '', 'posted': 'Posted February 3.', ..."
7,evcentric,"{'funny': '', 'posted': 'Posted December 4, 20..."
8,evcentric,"{'funny': '', 'posted': 'Posted November 3, 20..."
9,evcentric,"{'funny': '', 'posted': 'Posted October 15, 20..."


In [26]:
reviews = pd.json_normalize(data_reviews['reviews'])

In [27]:
reviews = reviews[['item_id', 'review']]

In [28]:
reviews.head(10)

,item_id,review
0,1250,Simple yet with great replayability. In my opi...
1,22200,It's unique and worth a playthrough.
2,43110,Great atmosphere. The gunplay can be a bit chu...
3,251610,I know what you think when you see this title ...
4,227300,For a simple (it's actually not all that simpl...
5,239030,Very fun little game to play when your bored o...
6,248820,A suitably punishing roguelike platformer. Wi...
7,370360,"""Run for fun? What the hell kind of fun is that?"""
8,237930,"Elegant integration of gameplay, story, world ..."
9,263360,"Random drops and random quests, with stat poin..."


In [29]:
reviews['review'] = reviews['review'].astype('str')

In [30]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analisis= SentimentIntensityAnalyzer()

def AnalisisSentimiento(text):
    sentimiento= analisis.polarity_scores(text)
    sentimiento= sentimiento['compound']
    return sentimiento
    
    

In [31]:
reviews['SentimientoReviews']= reviews['review'].apply(AnalisisSentimiento)

In [32]:
reviews['sentimiento'] = reviews['SentimientoReviews'].apply(lambda x: 0 if x < 0 else (1 if 0 <= x <= 0.3 else 2))

In [33]:
reviews['senti_negativo'] = reviews['SentimientoReviews'].apply(lambda x: 1 if x < 0 else 0)
reviews['senti_neutral'] = reviews['SentimientoReviews'].apply(lambda x: 1 if  0<= x <= 0.3 else 0)
reviews['senti_positivo'] = reviews['SentimientoReviews'].apply(lambda x: 1 if x > 0.3 else 0)

In [34]:
reviews.head(10)

,item_id,review,SentimientoReviews,sentimiento,senti_negativo,senti_neutral,senti_positivo
0,1250,Simple yet with great replayability. In my opi...,0.8481,2,0,0,1
1,22200,It's unique and worth a playthrough.,0.2263,1,0,1,0
2,43110,Great atmosphere. The gunplay can be a bit chu...,0.9117,2,0,0,1
3,251610,I know what you think when you see this title ...,0.9566,2,0,0,1
4,227300,For a simple (it's actually not all that simpl...,0.9708,2,0,0,1
5,239030,Very fun little game to play when your bored o...,0.7713,2,0,0,1
6,248820,A suitably punishing roguelike platformer. Wi...,0.3658,2,0,0,1
7,370360,"""Run for fun? What the hell kind of fun is that?""",0.3313,2,0,0,1
8,237930,"Elegant integration of gameplay, story, world ...",0.4767,2,0,0,1
9,263360,"Random drops and random quests, with stat poin...",0.0000,1,0,1,0


In [35]:
reviews= reviews[['item_id','senti_negativo','senti_neutral', 'senti_positivo']]

In [36]:
reviews.head(5)

,item_id,senti_negativo,senti_neutral,senti_positivo
0,1250,0,0,1
1,22200,0,1,0
2,43110,0,0,1
3,251610,0,0,1
4,227300,0,0,1


In [44]:
reviews = reviews.groupby('item_id').agg({
    'senti_negativo': 'sum',
    'senti_neutral': 'sum',
    'senti_positivo':'sum'}).reset_index()

In [45]:
reviews.head(5)

,item_id,senti_negativo,senti_neutral,senti_positivo
0,10,2,25,30
1,10090,6,7,39
2,10130,0,0,2
3,10140,1,0,0
4,10150,1,1,7


In [46]:
reviews.to_csv('C:/Users/HP/Documents/Proyectos/PI_1_Mlops/Data/Reviews.csv',index= False)

In [51]:
users = data_reviews
users= pd.json_normalize(data_reviews['reviews'])
users['user_id']= data_reviews['user_id']
users=users[['user_id','recommend']]

In [61]:
users['recommend']= users['recommend'].astype('str')

C:\Users\HP\AppData\Local\Temp\ipykernel_9896\794327691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['recommend']= users['recommend'].astype('str')


In [64]:
users['recommend_yes']= users['recommend'].apply(lambda x: 1 if x== 'True' else 0)
users['recommend_not']= users['recommend'].apply(lambda x: 1 if x== 'False' else 0)
users= users[['user_id', 'recommend_yes', 'recommend_not']]

C:\Users\HP\AppData\Local\Temp\ipykernel_9896\3448494662.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['recommend_yes']= users['recommend'].apply(lambda x: 1 if x== 'True' else 0)
C:\Users\HP\AppData\Local\Temp\ipykernel_9896\3448494662.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['recommend_not']= users['recommend'].apply(lambda x: 1 if x== 'False' else 0)


In [67]:
users = users.groupby('user_id').agg({
    'recommend_yes': 'sum',
    'recommend_not': 'sum'}).reset_index()

In [69]:
users.to_csv('C:/Users/HP/Documents/Proyectos/PI_1_Mlops/Data/Users.csv',index= False)

In [71]:
users.shape

(25485, 3)